In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np
import pandas as pd

In [ ]:
remote_data = xr.open_dataarray('http://tds.hycom.org/thredds/dodsC/GLBu0.08/expt_91.1/uv3z?water_u')
remote_data

In [ ]:
depths = np.array([0, 2, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50,
                   60, 70, 80, 90, 100, 125, 150, 200, 250, 300, 350, 400,
                   500, 600, 700, 800, 900, 1000, 1250, 1500, 2000, 2500,
                   3000, 4000, 5000])
# SS El Faro draft is approximately 13m. Average the data from the surface
# to 12 meters (0:6)
# https://en.wikipedia.org/wiki/SS_El_Faro
depth_dim = depths[0:6]
da = remote_data.isel(depth=slice(0, 6))

In [ ]:
# Find region of interest (30.5/-81.5/18/-65.5)
# make psudo lon and lat and coords aren't read in properly
plon = np.linspace(0,360-(360./len(da.lon)),num=len(da.lon))
plat = np.linspace(-80,80,num=len(da.lat))
lonw = np.abs(plon-(360-81.5)).argmin()
lone = np.abs(plon-(360-65.5)).argmin()
latn = np.abs(plat-30.5).argmin()
lats = np.abs(plat-18.).argmin()
latdim = plat[lats:latn]
londim = plon[lonw:lone]
print(lonw, lone, lats, latn)

In [ ]:
da2 = da.isel(lat=slice(lats, latn), lon=slice(lonw, lone))

In [ ]:
#da2.isel(time=0).plot()

In [ ]:
# make pseudo time as coords aren't read in
# Daily missing days
# https://hycom.org/faqs/463-nrl-netcdf-outputs-missing-days
missingdays = ['2015-09-19', '2015-03-25', '2015-03-15', '2015-01-02', '2014-04-13']
# First time point is 2014-04-07T00:00:00Z. Last time points is 2016-04-18T00:00:00Z
times = pd.date_range(start='04/07/2014',  end='04/18/2016', freq='D')
# Drop these time values
for i, t in enumerate(missingdays):
    _loc = times.get_loc(missingdays[i])
    times = times.drop(times[_loc])
    
# Select Sep 29th - October 3rd
ts = times.get_loc('2015-09-29')
te = times.get_loc('2015-10-03')+1
timedim = times[ts:te]
print(ts, te)

In [ ]:
dau = da2.isel(time=slice(ts, te))
dau_avg = dau.mean(dim='depth')
print(dau_avg)
dau_avg.isel(time=0).plot()

In [ ]:
remote_data = xr.open_dataarray('http://tds.hycom.org/thredds/dodsC/GLBu0.08/expt_91.1/uv3z?water_v')
dav = remote_data.isel(depth=slice(0, 6), lat=slice(lats, latn), lon=slice(lonw, lone), time=slice(ts, te))
dav_avg = dav.mean(dim='depth')
print(dav_avg)

In [ ]:
_dau = xr.DataArray(dau_avg.values, coords=[timedim, latdim, londim], dims=['time', 'lat', 'lon'])
_dav = xr.DataArray(dav_avg.values, coords=[timedim, latdim, londim], dims=['time', 'lat', 'lon'])
ds = xr.Dataset({'u': _dau, 'v': _dav})
ds.to_netcdf('HyCOM.nc')
print(ds)